In [13]:
#train the model and save it
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np
import csv

In [3]:
#form input_texts
input_texts = []
with open('test_data(copy).csv') as csvfile:
    next(csvfile)
    readCSV = csv.reader(csvfile,delimiter=',')
    for row in readCSV:
        text = " ".join(row)
        input_texts.append(text)
input_texts

['1 Debit 2339 India US USD 1 0.72',
 '2 Debit 4447 Pakistan Germany Euro 1 0.38',
 '3 Debit 6640 India India INR 0 0.91',
 '4 Debit 6076 Pakistan Japan Euro 0 0.41',
 '5 Debit 4790 US India INR 1 0.68',
 '6 Debit 7583 India India INR 1 0.09',
 '7 Credit 5915 Japan China Yen 1 0.89',
 '8 Debit 5733 Pakistan US USD 0 0.94',
 '9 Debit 776 India Germany INR 1 0.71',
 '10 Debit 8928 Pakistan India INR 1 0.37',
 '11 Credit 3372 India US USD 0 0.53',
 '12 Debit 4982 Pakistan Germany Euro 0 0.35',
 '13 Debit 8683 India India INR 1 0.25',
 '14 Credit 5369 Pakistan Japan Euro 0 0.29',
 '15 Credit 4708 US India INR 1 0.34',
 '16 Credit 5032 India India INR 0 0.65',
 '17 Credit 2872 Japan China Yen 1 0.72',
 '18 Credit 7435 Pakistan US USD 0 0.3',
 '19 Debit 3724 India Germany INR 1 0.41',
 '20 Credit 2923 Pakistan India INR 0 0.72',
 '21 Credit 7201 India US USD 1 0.23',
 '22 Credit 7390 Pakistan Germany Euro 1 0.72',
 '23 Debit 749 India India INR 1 0.86',
 '24 Debit 8942 Pakistan Japan Euro 0 

In [1]:
#form output_texts
with open('description.txt','r',encoding='utf-8') as f:
    lines =   f.read().split('\n') 
target_texts = []
for line in lines:
    line = 'START_ ' + line + ' _END'
    target_texts.append(line)
target_texts

['START_ Transaction no 1 is a debit transaction, from India to US worth 2339 dollars, which is not suspicious with 0.72 probability. _END',
 'START_ Transaction no 2 is a debit transaction, from Pakistan to Germany worth 4447 euros, which is not suspicious with 0.38 probability. _END',
 'START_ Transaction no 3 is a debit transaction, within India worth 6640 ruppees, which is suspicious with 0.91 probability. _END',
 'START_ Transaction no 4 is a debit transaction, from Pakistan to Japan worth 6076 euros, which is suspicious with 0.41 probability. _END',
 'START_ Transaction no 5 is a debit transaction, from US to India worth 4790 ruppees, which is not suspicious with 0.68 probability. _END',
 'START_ Transaction no 6 is a debit transaction, within India worth 7583 ruppees, which is not suspicious with 0.09 probability. _END',
 'START_ Transaction no 7 is a credit transaction, from Japan to China worth 5915 yen, which is not suspicious with 0.89 probability. _END',
 'START_ Transactio

In [5]:
input_words = set()
target_words = set()
for input_text,target_text in zip(input_texts,target_texts):
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)  

In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 10112
Number of unique output tokens: 10129
Max sequence length for inputs: 8
Max sequence length for outputs: 23


In [7]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [9]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]
    for t, word in enumerate(target_text.split(' ')):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word] 
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [10]:
embedding_size = 50
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [11]:
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     505600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     506450      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [14]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
checkpoint = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=250) #save model after every 250 epochs
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=1000,
          validation_split=0.2,callbacks=[checkpoint])
model.save('s2s.h5')
#training was stopped after around 500 epochs as loss value was low (0.056)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/1000
8000/8000 [==============================] - 26s 3ms/step - loss: 5.5431 - acc: 0.0929 - val_loss: 3.9546 - val_acc: 0.0870
Epoch 2/1000
8000/8000 [==============================] - 21s 3ms/step - loss: 3.8242 - acc: 0.1228 - val_loss: 3.7898 - val_acc: 0.1304
Epoch 3/1000
8000/8000 [==============================] - 22s 3ms/step - loss: 3.6465 - acc: 0.1304 - val_loss: 3.5699 - val_acc: 0.1304
Epoch 4/1000
8000/8000 [==============================] - 21s 3ms/step - loss: 3.3182 - acc: 0.2062 - val_loss: 3.1916 - val_acc: 0.2922
Epoch 5/1000
8000/8000 [==============================] - 21s 3ms/step - loss: 2.9360 - acc: 0.3752 - val_loss: 2.8502 - val_acc: 0.4534
Epoch 6/1000
8000/8000 [==============================] - 22s 3ms/step - loss: 2.6043 - acc: 0.5058 - val_loss: 2.5277 - val_acc: 0.5690
Epoch 7/1000
8000/8000 [===========

8000/8000 [==============================] - 17s 2ms/step - loss: 0.5474 - acc: 0.8189 - val_loss: 1.3711 - val_acc: 0.7839
Epoch 116/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5431 - acc: 0.8197 - val_loss: 1.3733 - val_acc: 0.7838
Epoch 117/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5401 - acc: 0.8201 - val_loss: 1.3792 - val_acc: 0.7838
Epoch 118/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5366 - acc: 0.8209 - val_loss: 1.3805 - val_acc: 0.7828
Epoch 119/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5325 - acc: 0.8223 - val_loss: 1.3854 - val_acc: 0.7838
Epoch 120/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5294 - acc: 0.8224 - val_loss: 1.3839 - val_acc: 0.7841
Epoch 121/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5264 - acc: 0.8237 - val_loss: 1.3948 - val_acc: 0.7830
Epoch 122/1000
8000/8000 [================

8000/8000 [==============================] - 17s 2ms/step - loss: 0.3764 - acc: 0.8556 - val_loss: 1.4779 - val_acc: 0.7847
Epoch 175/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3751 - acc: 0.8557 - val_loss: 1.4824 - val_acc: 0.7847
Epoch 176/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3715 - acc: 0.8570 - val_loss: 1.4864 - val_acc: 0.7841
Epoch 177/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3695 - acc: 0.8569 - val_loss: 1.4832 - val_acc: 0.7852
Epoch 178/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3672 - acc: 0.8577 - val_loss: 1.4811 - val_acc: 0.7855
Epoch 179/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3654 - acc: 0.8582 - val_loss: 1.4879 - val_acc: 0.7851
Epoch 180/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.3630 - acc: 0.8584 - val_loss: 1.4925 - val_acc: 0.7848
Epoch 181/1000
8000/8000 [================

8000/8000 [==============================] - 17s 2ms/step - loss: 0.2680 - acc: 0.8833 - val_loss: 1.5728 - val_acc: 0.7827
Epoch 234/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.2667 - acc: 0.8834 - val_loss: 1.5747 - val_acc: 0.7833
Epoch 235/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.2649 - acc: 0.8835 - val_loss: 1.5770 - val_acc: 0.7833
Epoch 236/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.2631 - acc: 0.8837 - val_loss: 1.5739 - val_acc: 0.7837
Epoch 237/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.2617 - acc: 0.8842 - val_loss: 1.5725 - val_acc: 0.7842
Epoch 238/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.2599 - acc: 0.8847 - val_loss: 1.5735 - val_acc: 0.7841
Epoch 239/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.2578 - acc: 0.8852 - val_loss: 1.5836 - val_acc: 0.7830
Epoch 240/1000
8000/8000 [================

/home/purva/anaconda3/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


...

Epoch 310/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.1731 - acc: 0.9029 - val_loss: 1.6482 - val_acc: 0.7832
Epoch 311/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1720 - acc: 0.9032 - val_loss: 1.6438 - val_acc: 0.7832
Epoch 312/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1709 - acc: 0.9035 - val_loss: 1.6478 - val_acc: 0.7827
Epoch 313/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1704 - acc: 0.9035 - val_loss: 1.6567 - val_acc: 0.7820
Epoch 314/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1692 - acc: 0.9036 - val_loss: 1.6462 - val_acc: 0.7832
Epoch 315/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1682 - acc: 0.9038 - val_loss: 1.6540 - val_acc: 0.7828
Epoch 316/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1673 - acc: 0.9037 - val_loss: 1.6521 - val_acc: 0.7827
Epoch 317/1000
8000/8000 [=

Epoch 369/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1226 - acc: 0.9103 - val_loss: 1.7048 - val_acc: 0.7806
Epoch 370/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.1223 - acc: 0.9103 - val_loss: 1.7004 - val_acc: 0.7814
Epoch 371/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.1215 - acc: 0.9104 - val_loss: 1.7003 - val_acc: 0.7812
Epoch 372/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.1198 - acc: 0.9109 - val_loss: 1.7011 - val_acc: 0.7814
Epoch 373/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1195 - acc: 0.9108 - val_loss: 1.6993 - val_acc: 0.7818
Epoch 374/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1195 - acc: 0.9106 - val_loss: 1.7090 - val_acc: 0.7805
Epoch 375/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1182 - acc: 0.9106 - val_loss: 1.7073 - val_acc: 0.7814
Epoch 376/1000
8000/8000 [=

Epoch 428/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0858 - acc: 0.9141 - val_loss: 1.7352 - val_acc: 0.7809
Epoch 429/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0857 - acc: 0.9139 - val_loss: 1.7361 - val_acc: 0.7805
Epoch 430/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0844 - acc: 0.9140 - val_loss: 1.7381 - val_acc: 0.7799
Epoch 431/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0841 - acc: 0.9142 - val_loss: 1.7397 - val_acc: 0.7800
Epoch 432/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0837 - acc: 0.9142 - val_loss: 1.7410 - val_acc: 0.7805
Epoch 433/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0841 - acc: 0.9139 - val_loss: 1.7432 - val_acc: 0.7796
Epoch 434/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0828 - acc: 0.9142 - val_loss: 1.7441 - val_acc: 0.7797
Epoch 435/1000
8000/8000 [=

Epoch 487/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.0598 - acc: 0.9158 - val_loss: 1.7742 - val_acc: 0.7785
Epoch 488/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.0601 - acc: 0.9156 - val_loss: 1.7722 - val_acc: 0.7798
Epoch 489/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.0594 - acc: 0.9157 - val_loss: 1.7754 - val_acc: 0.7796
Epoch 490/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.0593 - acc: 0.9157 - val_loss: 1.7805 - val_acc: 0.7789
Epoch 491/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0582 - acc: 0.9157 - val_loss: 1.7734 - val_acc: 0.7796
Epoch 492/1000
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0588 - acc: 0.9156 - val_loss: 1.7772 - val_acc: 0.7792
Epoch 493/1000
8000/8000 [==============================] - 18s 2ms/step - loss: 0.0584 - acc: 0.9157 - val_loss: 1.7747 - val_acc: 0.7797
Epoch 494/1000
8000/8000 [=

KeyboardInterrupt: 

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3336282307214785039
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5843726564242651578
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3634102272
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17515537537774843617
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8708373189746071517
physical_device_desc: "device: XLA_GPU device"
]
